In [1]:
from keras.initializers import Initializer
import numpy as np
import glob
import pandas as pd



In [2]:
class InitFromFile(Initializer):
    """ Initialize the weights by loading from file.

    # Arguments
        filename: name of file, should by .npy file
    """
    def __init__(self, filename):
        self.filename = filename

    def __call__(self, shape, dtype=None):
        with open(self.filename, "rb") as f:
            X = np.load(f)
        assert shape == X.shape
        return X

    def get_config(self):
        return {
            'filename': self.filename
        }

In [3]:
# from keras.initializers import Initializer
# from sklearn.cluster import KMeans


# class InitCentersKMeans(Initializer):
#     """ Initializer for initialization of centers of RBF network
#         by clustering the given data set.
#     # Arguments
#         X: matrix, dataset
#     """

#     def __init__(self, X, max_iter=100):
#         self.X = X
#         self.max_iter = max_iter

#     def __call__(self, shape, dtype=None):
#         assert shape[1] == self.X.shape[1]

#         n_centers = shape[0]
#         km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0)
#         km.fit(self.X)
#         return km.cluster_centers_


In [4]:
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras.initializers import RandomUniform, Initializer, Constant
import numpy as np


class InitCentersRandom(Initializer):
    """ Initializer for initialization of centers of RBF network
        as random samples from the given data set.

    # Arguments
        X: matrix, dataset to choose the centers from (random rows
          are taken as centers)
    """

    def __init__(self, X):
        self.X = X

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]
        idx = np.random.randint(self.X.shape[0], size=shape[0])

    # type checking to access elements of data correctly
        if type(self.X) == np.ndarray:
            return self.X[idx, :]
        elif type(self.X) == pd.core.frame.DataFrame:
            return self.X.iloc[idx, :]


class RBFLayer(Layer):
    """ Layer of Gaussian RBF units.

    # Example

    ```python
        model = Sequential()
        model.add(RBFLayer(10,
                           initializer=InitCentersRandom(X),
                           betas=1.0,
                           input_shape=(1,)))
        model.add(Dense(1))
    ```


    # Arguments
        output_dim: number of hidden units (i.e. number of outputs of the
                    layer)
        initializer: instance of initiliazer to initialize centers
        betas: float, initial value for betas

    """

    def __init__(self, output_dim, initializer=None, betas=1.0, **kwargs):
        self.output_dim = output_dim
        self.init_betas = betas
        if not initializer:
            self.initializer = RandomUniform(0.0, 1.0)
        else:
            self.initializer = initializer
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.output_dim,),
                                     initializer=Constant(
                                         value=self.init_betas),
                                     # initializer='ones',
                                     trainable=True)

        super(RBFLayer, self).build(input_shape)

    def call(self, x):

        C = K.expand_dims(self.centers)
        H = K.transpose(C-K.transpose(x))
        return K.exp(-self.betas * K.sum(H**2, axis=1))

        # C = self.centers[np.newaxis, :, :]
        # X = x[:, np.newaxis, :]

        # diffnorm = K.sum((C-X)**2, axis=-1)
        # ret = K.exp( - self.betas * diffnorm)
        # return ret

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        # have to define get_config to be able to use model_from_json
        config = {
            'output_dim': self.output_dim
        }
        base_config = super(RBFLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt


def load_data():

    data = np.loadtxt("C:/Users/89344/Downloads/rbf_keras-master/rbf_keras-master/data/data.txt")
    X = data[:, :-1]
    y = data[:, -1:]
    return X, y




In [6]:


path = 'C:/Users/89344/Desktop/人工神经网络/vowel'
file_name = glob.glob(path+'/*')
data = []
for i in file_name:
    data.append(pd.read_table(i, sep = '\t', header = 0))
train_data = data[1]
test_data = data[0]
train_data_string =  train_data.loc[0]
train_data.drop(labels=[0,1],inplace = True)
train_data.reset_index(drop = True,inplace = True)
test_data_string =  test_data.loc[0]
test_data.drop(labels=[0,1],inplace = True)
test_data.reset_index(drop = True,inplace = True)
#lst = ['x.1']
lst = ['x.1', 'x.2', 'x.3', 'x.4', 'x.5', 'x.6', 'x.7','x.8', 'x.9', 'x.10']
X = np.array(train_data[lst]).astype(float)
y = np.array(train_data['y']).astype(float)
X_test = np.array(test_data[lst]).astype(float)
y_test = np.array(test_data['y']).astype(float)

model = Sequential()
rbflayer = RBFLayer(10,
                    initializer=InitCentersRandom(X),
                    betas=2.0,
                    input_shape=(10,))
model.add(rbflayer)
model.add(Dense(1))

model.compile(loss='mean_squared_error',
              optimizer=RMSprop())


In [7]:
model.fit(X, y,
          batch_size=50,
          epochs=2000,
          verbose=1)

Epoch 1/2000
11/11 [==============================] - 0s 1ms/step - loss: 45.8831
Epoch 2/2000
11/11 [==============================] - 0s 900us/step - loss: 45.6951
Epoch 3/2000
11/11 [==============================] - 0s 1ms/step - loss: 45.5408
Epoch 4/2000
11/11 [==============================] - 0s 1ms/step - loss: 45.3931
Epoch 5/2000
11/11 [==============================] - 0s 1ms/step - loss: 45.2486
Epoch 6/2000
11/11 [==============================] - 0s 950us/step - loss: 45.1055
Epoch 7/2000
11/11 [==============================] - 0s 950us/step - loss: 44.9621
Epoch 8/2000
11/11 [==============================] - 0s 900us/step - loss: 44.8185
Epoch 9/2000
11/11 [==============================] - 0s 950us/step - loss: 44.6751
Epoch 10/2000
11/11 [==============================] - 0s 1ms/step - loss: 44.5325
Epoch 11/2000
11/11 [==============================] - 0s 1ms/step - loss: 44.3909
Epoch 12/2000
11/11 [==============================] - 0s 1ms/step - loss: 44.2479
Epo

11/11 [==============================] - 0s 900us/step - loss: 32.5208
Epoch 99/2000
11/11 [==============================] - 0s 850us/step - loss: 32.3608
Epoch 100/2000
11/11 [==============================] - 0s 850us/step - loss: 32.2010
Epoch 101/2000
11/11 [==============================] - 0s 1ms/step - loss: 32.0378
Epoch 102/2000
11/11 [==============================] - 0s 950us/step - loss: 31.8746
Epoch 103/2000
11/11 [==============================] - 0s 800us/step - loss: 31.7086
Epoch 104/2000
11/11 [==============================] - 0s 900us/step - loss: 31.5388
Epoch 105/2000
11/11 [==============================] - 0s 950us/step - loss: 31.3680
Epoch 106/2000
11/11 [==============================] - 0s 950us/step - loss: 31.1939
Epoch 107/2000
11/11 [==============================] - 0s 900us/step - loss: 31.0168
Epoch 108/2000
11/11 [==============================] - 0s 850us/step - loss: 30.8346
Epoch 109/2000
11/11 [==============================] - 0s 950us/step - 

11/11 [==============================] - 0s 850us/step - loss: 5.6627
Epoch 195/2000
11/11 [==============================] - 0s 850us/step - loss: 5.5951
Epoch 196/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.5292
Epoch 197/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.4651
Epoch 198/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.4092
Epoch 199/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.3564
Epoch 200/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.3069
Epoch 201/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.2563
Epoch 202/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.2089
Epoch 203/2000
11/11 [==============================] - 0s 900us/step - loss: 5.1725
Epoch 204/2000
11/11 [==============================] - 0s 950us/step - loss: 5.1219
Epoch 205/2000
11/11 [==============================] - 0s 1ms/step - loss: 5.0894
Epoch 206/2

11/11 [==============================] - 0s 950us/step - loss: 3.5551
Epoch 292/2000
11/11 [==============================] - 0s 950us/step - loss: 3.5371
Epoch 293/2000
11/11 [==============================] - 0s 1ms/step - loss: 3.5273
Epoch 294/2000
11/11 [==============================] - 0s 950us/step - loss: 3.5210
Epoch 295/2000
11/11 [==============================] - 0s 900us/step - loss: 3.5151
Epoch 296/2000
11/11 [==============================] - 0s 850us/step - loss: 3.5008
Epoch 297/2000
11/11 [==============================] - 0s 1ms/step - loss: 3.4874
Epoch 298/2000
11/11 [==============================] - 0s 1ms/step - loss: 3.4869
Epoch 299/2000
11/11 [==============================] - 0s 900us/step - loss: 3.4806
Epoch 300/2000
11/11 [==============================] - 0s 800us/step - loss: 3.4629
Epoch 301/2000
11/11 [==============================] - 0s 950us/step - loss: 3.4546
Epoch 302/2000
11/11 [==============================] - 0s 1ms/step - loss: 3.4492
Epo

11/11 [==============================] - 0s 750us/step - loss: 2.8403
Epoch 388/2000
11/11 [==============================] - 0s 900us/step - loss: 2.8339
Epoch 389/2000
11/11 [==============================] - 0s 800us/step - loss: 2.8318
Epoch 390/2000
11/11 [==============================] - 0s 850us/step - loss: 2.8181
Epoch 391/2000
11/11 [==============================] - 0s 850us/step - loss: 2.8201
Epoch 392/2000
11/11 [==============================] - 0s 1ms/step - loss: 2.8129
Epoch 393/2000
11/11 [==============================] - 0s 1ms/step - loss: 2.8080
Epoch 394/2000
11/11 [==============================] - 0s 900us/step - loss: 2.7939
Epoch 395/2000
11/11 [==============================] - 0s 900us/step - loss: 2.7955
Epoch 396/2000
11/11 [==============================] - 0s 850us/step - loss: 2.7969
Epoch 397/2000
11/11 [==============================] - 0s 950us/step - loss: 2.7856
Epoch 398/2000
11/11 [==============================] - 0s 950us/step - loss: 2.7861

...

11/11 [==============================] - 0s 1ms/step - loss: 1.1087
Epoch 1543/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.1025
Epoch 1544/2000
11/11 [==============================] - 0s 950us/step - loss: 1.1074
Epoch 1545/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.1000
Epoch 1546/2000
11/11 [==============================] - 0s 900us/step - loss: 1.1042
Epoch 1547/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0991
Epoch 1548/2000
11/11 [==============================] - 0s 950us/step - loss: 1.1020
Epoch 1549/2000
11/11 [==============================] - 0s 850us/step - loss: 1.0990
Epoch 1550/2000
11/11 [==============================] - 0s 900us/step - loss: 1.1049
Epoch 1551/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0982
Epoch 1552/2000
11/11 [==============================] - 0s 900us/step - loss: 1.0952
Epoch 1553/2000
11/11 [==============================] - 0s 900us/step - loss: 1

11/11 [==============================] - 0s 900us/step - loss: 1.0460
Epoch 1639/2000
11/11 [==============================] - 0s 950us/step - loss: 1.0578
Epoch 1640/2000
11/11 [==============================] - 0s 800us/step - loss: 1.0524
Epoch 1641/2000
11/11 [==============================] - 0s 900us/step - loss: 1.0522
Epoch 1642/2000
11/11 [==============================] - 0s 950us/step - loss: 1.0463
Epoch 1643/2000
11/11 [==============================] - 0s 900us/step - loss: 1.0507
Epoch 1644/2000
11/11 [==============================] - 0s 850us/step - loss: 1.0498
Epoch 1645/2000
11/11 [==============================] - 0s 850us/step - loss: 1.0498
Epoch 1646/2000
11/11 [==============================] - 0s 950us/step - loss: 1.0527
Epoch 1647/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0480
Epoch 1648/2000
11/11 [==============================] - 0s 900us/step - loss: 1.0523
Epoch 1649/2000
11/11 [==============================] - 0s 1ms/step - l

11/11 [==============================] - 0s 900us/step - loss: 1.0033
Epoch 1735/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0046
Epoch 1736/2000
11/11 [==============================] - 0s 950us/step - loss: 0.9999
Epoch 1737/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0072
Epoch 1738/2000
11/11 [==============================] - 0s 900us/step - loss: 1.0079
Epoch 1739/2000
11/11 [==============================] - 0s 950us/step - loss: 0.9987
Epoch 1740/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0097
Epoch 1741/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0002
Epoch 1742/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0072
Epoch 1743/2000
11/11 [==============================] - 0s 950us/step - loss: 1.0020
Epoch 1744/2000
11/11 [==============================] - 0s 1ms/step - loss: 1.0033
Epoch 1745/2000
11/11 [==============================] - 0s 950us/step - loss: 1.0

11/11 [==============================] - 0s 1ms/step - loss: 0.9631
Epoch 1831/2000
11/11 [==============================] - 0s 1ms/step - loss: 0.9628
Epoch 1832/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9620
Epoch 1833/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9635
Epoch 1834/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9596
Epoch 1835/2000
11/11 [==============================] - 0s 950us/step - loss: 0.9614
Epoch 1836/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9564
Epoch 1837/2000
11/11 [==============================] - 0s 950us/step - loss: 0.9603
Epoch 1838/2000
11/11 [==============================] - 0s 1ms/step - loss: 0.9579
Epoch 1839/2000
11/11 [==============================] - 0s 850us/step - loss: 0.9641
Epoch 1840/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9569
Epoch 1841/2000
11/11 [==============================] - 0s 900us/step - los

11/11 [==============================] - 0s 900us/step - loss: 0.9234
Epoch 1927/2000
11/11 [==============================] - 0s 850us/step - loss: 0.9150
Epoch 1928/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9284
Epoch 1929/2000
11/11 [==============================] - 0s 1ms/step - loss: 0.9241
Epoch 1930/2000
11/11 [==============================] - 0s 1ms/step - loss: 0.9192
Epoch 1931/2000
11/11 [==============================] - 0s 1ms/step - loss: 0.9128
Epoch 1932/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9186
Epoch 1933/2000
11/11 [==============================] - 0s 1ms/step - loss: 0.9179
Epoch 1934/2000
11/11 [==============================] - 0s 850us/step - loss: 0.9212
Epoch 1935/2000
11/11 [==============================] - 0s 1ms/step - loss: 0.9126
Epoch 1936/2000
11/11 [==============================] - 0s 900us/step - loss: 0.9181
Epoch 1937/2000
11/11 [==============================] - 0s 850us/step - loss: 0

In [8]:
y_pred = model.predict(X_test)
print(rbflayer.get_weights())






15/15 [==============================] - 0s 929us/step
[array([[-2.9926322 ,  1.0883691 ,  0.28420857, -0.17942597, -0.5901485 ,
         1.2815492 , -0.11697263,  0.20997775,  0.44757444, -0.05604247],
       [-5.192979  ,  2.96735   ,  0.6643824 , -0.5691359 ,  0.99632645,
         0.21309225, -0.18881951,  0.9444706 , -0.9640254 , -0.2812156 ],
       [-2.7204282 ,  3.754502  , -0.40277043,  0.05459117, -1.0971756 ,
         0.2065536 , -0.2644672 ,  2.0980546 ,  0.11593883, -0.4290023 ],
       [-2.7538228 ,  1.5022876 , -0.08642156, -0.30423132,  0.37893236,
        -0.05903376,  0.07066691,  0.32485682, -1.5796354 ,  0.11395407],
       [-2.7784288 ,  2.4371676 , -0.64887017,  0.47131234, -0.6961843 ,
         0.31552124, -0.32823834,  0.30406025,  0.07609267, -0.36430573],
       [-2.828811  ,  1.4055073 , -1.1628854 ,  0.9912413 , -0.74604285,
         1.0925248 , -0.1633855 ,  1.1679637 ,  0.03395826, -0.9097329 ],
       [-3.6216736 ,  2.3559306 , -0.22721834,  1.3945837 , -1